In [1]:
import netCDF4 as nc
import numpy as np
import os, sys
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pyproj import Proj
import numpy as np


from fun_goes_process import *

In [2]:
ind1,ind2,ind3,ind4 = 829,832, 1713,1716

In [3]:
file_path = './GOES_data/GOES-17/ABI-L1b-RadF/202116020/'
Fls = os.listdir(file_path)
Fls

['ABI-L1b-RadF_2021_160_20_OR_ABI-L1b-RadF-M6C02_G17_s20211602050321_e20211602059388_c20211602059413.nc',
 'ABI-L1b-RadF_2021_160_20_OR_ABI-L1b-RadF-M6C03_G17_s20211602050321_e20211602059388_c20211602059419.nc',
 'ABI-L1b-RadF_2021_160_20_OR_ABI-L1b-RadF-M6C04_G17_s20211602050321_e20211602059388_c20211602059408.nc',
 'ABI-L1b-RadF_2021_160_20_OR_ABI-L1b-RadF-M6C05_G17_s20211602050321_e20211602059388_c20211602059432.nc',
 'ABI-L1b-RadF_2021_160_20_OR_ABI-L1b-RadF-M6C06_G17_s20211602050321_e20211602059394_c20211602059425.nc',
 'ABI-L1b-RadF_2021_160_20_OR_ABI-L1b-RadF-M6C01_G17_s20211602050321_e20211602059388_c20211602059423.nc']

In [78]:
file_path = './GOES_data/GOES-17/ABI-L1b-RadF/202116020/'
Fls = os.listdir(file_path)
dataset = nc.Dataset(file_path+Fls[0], mode='r')
Fls[0]

'ABI-L1b-RadF_2021_160_20_OR_ABI-L1b-RadF-M6C02_G17_s20211602050321_e20211602059388_c20211602059413.nc'

In [85]:
Rad_data.shape

(21696, 21696)

In [96]:
i=0
Rad_data = dataset.variables['Rad']
ds_f = Rad_data.shape[0] // 2701 
# downsample to the grid 4km
if ds_f > 1:
    print(ds_f)
    # Method 1: No average between grids
    # rad = Rad_data[::p, ::p]
    # Method 2: Average between grids
    rad = np.mean(
        Rad_data[:].reshape(nrows // ds_f, ds_f, 
                        ncols // ds_f, ds_f), 
                        axis=(1, 3)
    )
    y = dataset.variables['y'][::ds_f]
    x = dataset.variables['x'][::ds_f]
else:
    rad = Rad_data[:]
    y = dataset.variables['y']
    x = dataset.variables['x']
# extract the radiance data
rad_chx = rad[ind1:ind2, ind3:ind4][0,0]
ch_name = Fls[i][43:46]
# convert to utc
t_ = dataset.variables['t'][:]
utc_time = convert_to_utc(t_.item())

8
UTC Time: 2021-06-09 20:55:05.522566
676544105.522566
(Eastern Time): 2021-06-09 08:55:05.522566-04:00


In [98]:
# Calculate original indices from downsampled indices
yo = y[ind1]  # Calculate the original row index
xo = x[ind3]  # Calculate the original column index
# official constant
r_eq = 6378137  # semi-major axis in meters
r_pol = 6356752.31414   # semi-minor axis in meters
H = 42164160  # perspective point height + semi-major axis
# r_eq = dataset.variables['goes_imager_projection'].semi_major_axis  # m
# r_pol = dataset.variables['goes_imager_projection'].semi_minor_axis # m
# h0 = dataset.variables['goes_imager_projection'].perspective_point_height # m
# H = h0 + r_eq
lon_0 = dataset.variables['goes_imager_projection'].longitude_of_projection_origin # degree
# print(r_eq, r_pol, H, lon_0)
# important notice, official guide mark all grids in (y,x)
# rad[ind1:ind2, ind3:ind4].filled()[0,0] = [ind1,ind3] in grids-> y[ind1],x[ind3]
lat,lon=calculate_geodetic_coordinates(yo,xo, r_eq, r_pol, H, np.deg2rad(lon_0))
# Print results
print(f"Geodetic Latitude: {math.degrees(lat):.6f} degrees")
print(f"Geodetic Longitude: {math.degrees(lon):.6f} degrees")

1.0000234134143744 -84114198.86048532 1737135756914831
36432797.001904085 36340289.146776 -1456595.367130704 2147188.37448294
Geodetic Latitude: 19.558903 degrees
Geodetic Longitude: -122.957945 degrees


#  Geodetic Lat,Lon 
from N/S elevation angle (y) and E/W elevation angle (x)

## official user guide for lon,lat from xy

In [73]:
import numpy as np

def calculate_geodetic_coordinates(y, x, r_eq, r_pol, H, lon_0):
    """
    Calculate geodetic latitude and longitude from fixed grid coordinates
    
    Parameters:
    y: N/S Grid Fixed Scanning Angle (in radians)
    x: E/W Grid Fixed Scanning Angle (in radians)
    r_eq: Equatorial radius (semi-major axis) in meters
    r_pol: Polar radius (semi-minor axis) in meters
    H: Perspective point height + semi-major axis in meters
    lon_0: Longitude of projection origin (in radians)
    
    Returns:
    Tuple of (geodetic latitude, geodetic longitude) in radians
    """
    # Calculate intermediate values
    a = np.sin(x)**2 + np.cos(x)**2 * (np.cos(y)**2 + (r_eq**2 / r_pol**2) * np.sin(y)**2)
    b = -2 * H * np.cos(x) * np.cos(y)
    c = (H+r_eq)*(H-r_eq)
    print(a, b, c)
    # Calculate satellite distance from the point
    rs = (-b - np.sqrt(b**2 - 4 * a * c)) / (2 * a)
    
    # Calculate S_x, S_y, S_z components
    sx = rs * np.cos(x) * np.cos(y)
    sy = -rs * np.sin(x)
    sz = rs * np.cos(x) * np.sin(y)
    print(rs, sx, sy, sz)
    # Calculate geodetic latitude
    lat = np.arctan((r_pol**2 / r_eq**2) * sz / np.sqrt((H - sx)**2 + sy**2))
    
    # Calculate geodetic longitude
    lon = lon_0 - np.arctan2(sy, H - sx)
    
    return lat, lon


In [47]:
# official case
r_eq = 6378137  # semi-major axis in meters
r_pol = 6356752.31414   # semi-minor axis in meters
H = 42164160  # perspective point height + semi-major axis
lon_0 = -1.308996939  # longitude of projection origin

# Fixed grid coordinates from the example
y = 0.095340  # N/S Scanning Angle
x = -0.024052  # E/W Scanning Angle

# Calculate geodetic coordinates
lat, lon = calculate_geodetic_coordinates(y, x, r_eq, r_pol, H, lon_0)

# Print results
print(f"Geodetic Latitude: {lat:.6f} degrees")
print(f"Geodetic Longitude: {lon:.6f} degrees")

# Print results
print(f"Geodetic Latitude: {math.degrees(lat):.6f} degrees")
print(f"Geodetic Longitude: {math.degrees(lon):.6f} degrees")

1.0000610393966292 -83921070.03211282 1737135756914831
37116295.874486506 36937048.733040124 892635.0778582101 3532287.212730611
Geodetic Latitude: 0.584529 degrees
Geodetic Longitude: -1.478136 degrees
Geodetic Latitude: 33.491031 degrees
Geodetic Longitude: -84.690932 degrees


## pyproj

In [99]:
# google commity
# Satellite height
r_eq = 6378137  # semi-major axis in meters
r_pol = 6356752.31414   # semi-minor axis in meters
H = 42164160  # perspective point height + semi-major axis
lon_0 = -1.308996939  # longitude of projection origin

# Fixed grid coordinates from the example
y = 0.095340  # N/S Scanning Angle
x = -0.024052  # E/W Scanning Angle

sat_h = 35786023 #dataset.variables['goes_imager_projection'].perspective_point_height
# Satellite longitude
sat_lon = np.rad2deg(lon_0) #dataset.variables['goes_imager_projection'].longitude_of_projection_origin
# Satellite sweep
sat_sweep = dataset.variables['goes_imager_projection'].sweep_angle_axis # E-W:x
# The projection x and y coordinates equals
# the scanning angle (in radians) multiplied by the satellite height
# (https://proj.org/en/stable/operations/projections/geos.html)
X = x * sat_h
Y = y * sat_h
r_eq = 6378137.0
# map object with pyproj : https://proj.org/en/stable/usage/projections.html
p = Proj(proj='geos', h=sat_h, lon_0=sat_lon, sweep=sat_sweep, a=r_eq, b=r_pol, x_0=0, y_0=0, units='m', ellps='GRS80')
# Convert map points to latitude and longitude with the magic provided by Pyproj
XX, YY = np.meshgrid(X, Y)
lons, lats = p(XX, YY, inverse=True)  # 14.44 kilometers.
lons, lats
# utc_time = datetime(2018, 9, 5, 22, 30)
# sun_zenith = np.zeros((data4.shape[0], data4.shape[1]))
# sun_zenith = astronomy.sun_zenith_angle(utc_time, lons, lats)

(array([[-84.69093212]]), array([[33.84616229]]))

In [22]:
-84.690932, 33.491031

(-84.690932, 33.491031)

## different pyproj with official

In [102]:
import math

def haversine_distance(coord1, coord2):
    # The radius of the Earth in kilometers
    R = 6371.0
    
    # Unpack the coordinates
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    
    # Convert degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    
    # Differences in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Location coordinates
loc1_offi = (-84.690932, 33.491031)
loc2_proj = (-84.690932, 33.84616229)

# Calculate the distance
distance = haversine_distance(loc1_offi, loc2_proj)
print(f"The distance between the two locations is {distance:.2f} kilometers.")

The distance between the two locations is 39.49 kilometers.
